In [1]:
import numpy as np
import pandas as pd

In [61]:
class missing_filler_category():
    
    """
    Returns a table with all the missing values filled with special category.
    params::x DataFrame with the data
    params::filling_category a value to fill the missing values
    """
    
    def __init__(self, x, filling_category = "filler", y = None):
        self.x = x
        self.y = y
        self.filling_category = filling_category
        
    def fit(self):
        return self
    
    def transform(self):
        self.x.fillna(self.filling_category, inplace = True)
        filled_table = self.x
        return filled_table
    
    
class missing_filler_mean():
    
    """
    Returns a table with all the missing values in numerical columns filled with mean and all the 
    missing values in categorical columns filled with special category.
    params::x DataFrame with the data
    params::categorical_variables list of categorical varaibles
    params::filling_category a value to fill the missing values in categorical variables
    """
    
    def __init__(self, x, categorical_variables = [], filling_category = "filler", y = None):
        self.x = x
        self.y = y
        self.categorical_variables = categorical_variables
        self.filling_category = filling_category
        
    def fit(self):
        self.mean_to_fill = self.x.mean()
        return self
    
    def transform(self):
        filled_table = self.x.fillna(self.mean_to_fill)
        if len(self.categorical_variables) != 0:
            filled_table[self.categorical_variables].fillna(self.filling_category, inplace = True)
        return filled_table
    
    


Проверим, как работает

In [53]:
test_df = pd.DataFrame({"SomeVariable" : [1,2,4,5,None], 
                       "OneMoreVariable" : [12, 66, 12, None, 99], 
                       "SomeCategoricalVariable" : ["Biba", "I", "Boba", "Dva", None]})
test_df

,SomeVariable,OneMoreVariable,SomeCategoricalVariable
0,1.0,12.0,Biba
1,2.0,66.0,I
2,4.0,12.0,Boba
3,5.0,NaN,Dva
4,NaN,99.0,None


In [62]:
obj = missing_filler_category(test_df, "Grazhdanina")
obj.fit()
obj.transform()

,SomeVariable,OneMoreVariable,SomeCategoricalVariable
0,1.0,12.0,Biba
1,2.0,66.0,I
2,4.0,12.0,Boba
3,5.0,Grazhdanina,Dva
4,Grazhdanina,99.0,Grazhdanina


In [50]:
obj = missing_filler_mean(test_df, "SomeCategoricalVariable", "Grazhdanina")
obj.fit()
obj.transform()

,SomeVariable,OneMoreVariable,SomeCategoricalVariable
0,1.0,12.0,Biba
1,2.0,66.0,I
2,4.0,12.0,Boba
3,5.0,Grazhdanina,Dva
4,Grazhdanina,99.0,Grazhdanina
